# WikiStory 


In [1]:
import pandas as pd

### Convert to parquet

First a converesion from csv is parquet makes everything faster. If the file is in place, you don't need to run it again.

In [ ]:
csv_data = pd.read_csv('data.csv', quotechar ='|')
csv_data.to_parquet('data/input.parquet', engine='pyarrow')

### Add days to parquet
We need to add days to our parquet as that's how we will do clustering of words for later rendering

In [ ]:
input_data = pd.read_parquet('data/input.parquet', engine='pyarrow')

input_data['timestamp'] = pd.to_datetime(input_data['timestamp'])
input_data['day'] = input_data.timestamp.dt.date

In [ ]:
print(input_data.head())

In [ ]:
input_data.to_parquet('data/with_date.parquet',engine='pyarrow')

### Cleanup and Transformation

In [ ]:
with_dates = pd.read_parquet('data/with_date.parquet')
# with_dates = with_dates[with_dates.page_title != 'Tartışma:Anasayfa']
grouped_by_date_and_title = with_dates.groupby(['day','page_title']).count()
grouped_by_date_and_title

### Save Results as Parquet

In [2]:
grouped_by_date_and_title.to_parquet('data/grouped.parquet', engine='pyarrow')

NameError: name 'grouped_by_date_and_title' is not defined

### Save daily CSV's

In [2]:
grouped_by_date_and_title = pd.read_parquet('data/grouped.parquet')

In [57]:
more_than_two = grouped_by_date_and_title[grouped_by_date_and_title['page_id']>3]
more_than_two = more_than_two.reset_index()
grouped_by_day = more_than_two.groupby('day')
grouped_by_day.head()

,day,page_title,page_id,page_ns,revision_id,timestamp,contributor_id,contributor_name,bytes
0,2003-07-05,Anasayfa,6,6,6,6,6,6,6
1,2003-08-02,Karl Marx,4,4,4,4,4,4,4
2,2003-08-02,Linux,5,5,5,5,5,5,5
3,2003-09-05,Anasayfa,9,9,9,9,9,9,9
4,2003-11-18,Anasayfa,12,12,12,12,12,12,12
...,...,...,...,...,...,...,...,...,...
601665,2020-03-01,2019-20 1. Lig,4,4,4,4,4,4,4
601666,2020-03-01,2019-20 Süper Lig,15,15,15,15,15,15,15
601667,2020-03-01,2019-20 Süper Lig maçları,4,4,4,4,4,4,4
601668,2020-03-01,2019-2020 koronavirüs salgını,5,5,5,5,5,5,5


In [154]:
import concurrent.futures
import json
import numpy as np
import datetime

def write_head(arg):
    index = arg[0]
    group = arg[1]
    filename = index
    # tags = group.apply(lambda x: {x.page_id})
    df = group.rename(columns={"page_id": "revisions", "page_title":"title"})
    df = df[["title", "revisions"]]
    df.to_json(f'data/by_date_amcharts/{index}.json', orient='records')
#with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
#    executor.map(write_head, grouped_by_day)
    
dates = list(map(lambda x: x[0].strftime("%Y-%M-%d"), grouped_by_day.day))
    
index_dict = {'dates': dates}
with open('data/by_date_amcharts/index.json',"w+") as outfile:
  json.dump(index_dict, outfile)

# json.dumps(index_dict)
index_dict

{'dates': ['2003-00-05',
  '2003-00-02',
  '2003-00-05',
  '2003-00-18',
  '2003-00-05',
  '2003-00-08',
  '2003-00-24',
  '2004-00-02',
  '2004-00-07',
  '2004-00-08',
  '2004-00-11',
  '2004-00-13',
  '2004-00-14',
  '2004-00-15',
  '2004-00-16',
  '2004-00-19',
  '2004-00-20',
  '2004-00-22',
  '2004-00-23',
  '2004-00-25',
  '2004-00-26',
  '2004-00-29',
  '2004-00-30',
  '2004-00-01',
  '2004-00-04',
  '2004-00-06',
  '2004-00-09',
  '2004-00-11',
  '2004-00-20',
  '2004-00-21',
  '2004-00-22',
  '2004-00-02',
  '2004-00-05',
  '2004-00-06',
  '2004-00-07',
  '2004-00-11',
  '2004-00-13',
  '2004-00-16',
  '2004-00-18',
  '2004-00-21',
  '2004-00-24',
  '2004-00-25',
  '2004-00-28',
  '2004-00-02',
  '2004-00-03',
  '2004-00-04',
  '2004-00-05',
  '2004-00-11',
  '2004-00-12',
  '2004-00-13',
  '2004-00-14',
  '2004-00-15',
  '2004-00-17',
  '2004-00-18',
  '2004-00-20',
  '2004-00-22',
  '2004-00-23',
  '2004-00-26',
  '2004-00-27',
  '2004-00-28',
  '2004-00-29',
  '2004-00-30',